# LBT 2025A Proposal
* Prioritize targets that already have PEPSI spectra
  * Hope is to obtain enough measurements to characterize the orbits of some systems

In [1]:
from functools import partial
from sqlite3 import connect
import sys
import xml.etree.ElementTree as ET

import astroplan as ap
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
import numpy as np
import pandas as pd

sys.path.append("../../")
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt

In [2]:
# prepare list of all targets
conn = connect("../../django/TargetDB/db.sqlite3")

observer = ap.Observer.at_site("LBT", timezone="US/Arizona")
semester = (Time("2025-01-01"), Time("2025-06-30"))

creator = tlc.TargetListCreator(connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="tess"),
    tlc.add_lists,
    tlc.add_speckle,
    tlc.add_spectra,
    partial(tlc.add_observability, observer=observer, time_segment=semester),
    partial(lbt.add_pepsi_params, fiber="300", cd_blue=3, cd_red=6, snr=100, binocular=True),
]

targets = creator.calculate()

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4620: RuntimeWarning: invalid value encountered in ld
  p1 = ufunc.ld(bm, p, q, e, em, dlim)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:19004: RuntimeWarning: invalid value encountered in anp
  c_retval = ufunc.anp(a)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4036: RuntimeWarning: invalid value encountered in atioq
  aob, zob, hob, dob, rob = ufunc.atioq(ri, di, astrom)


In [3]:
# filter all targets into ones we want for this proposal
criteria = (
    True
    # & (targets["list_LBT_2024B"])
    & (targets["pepsi_count"] > 0)
    & (targets["observable"])
)
lbt_targets = targets[criteria]

# double the exposure time as a safety factor on the snr
lbt_targets.loc[:, "pepsi_exp_time"] *= 2

# add in overhead to each target
lbt_targets.loc[:, "pepsi_exp_time"] += 120

# pare down to only what we need, and rename to match PIT expectations
columns = {
    "target_name": "Name",
    "ra": "RAJ2000",
    "dec": "DecJ2000",
    "pmra": "pmRA",
    "pmdec": "pmDec",
    "Vmag": "V",
}
pit_list = lbt_targets[list(columns.keys())]
pit_list = pit_list.rename(columns=columns)

pit_list.to_csv("LBT 2025A.csv")
print(f"{np.sum(lbt_targets["pepsi_exp_time"]) / 3600:.2f} hours observing time")
Table.from_pandas(lbt_targets)#.pprint_all()

2.69 hours observing time


target_name,target_source,target_type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,RA,Dec,list_Featured_targets,list_Gemini_2024B_Proposal,list_Gemini_2025A_Proposal,list_HQND,list_Kostov_2022,list_Kostov_2023,list_LBT_2023-12-06,list_LBT_2023B,list_LBT_2024B,list_Speckle_Resolved,speckle_count,pepsi_count,observable,pepsi_fiber,pepsi_cd_blue,pepsi_cd_blue_num_exp,pepsi_cd_red,pepsi_cd_red_num_exp,pepsi_snr,pepsi_exp_time,pepsi_priority,pepsi_notes
str13,str28,str6,float64,float64,float64,float64,float64,float64,float64,str11,str12,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,int64,int64,bool,str3,int64,int64,int64,int64,int64,float64,str1,str1
TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,223.425163143389,52.7158479482434,2.17801,-3.59429,2.02761,12.497,5860.0,14:53:42.04,+52:42:57.05,False,False,False,False,True,False,False,True,True,True,18,2,True,300,3,1,6,1,100,565.0,,
TIC 367448265,Kostov 2022 arXiv:2202.05790,QuadEB,78.3824380721019,35.6530529938371,-5.85675,-3.43287,3.26415,7.9,9212.0,5:13:31.79,+35:39:10.99,False,False,False,False,True,False,True,True,True,True,3,2,True,300,3,1,6,1,100,132.8,,
TIC 25818450,Kostov 2022 arXiv:2202.05790,QuadEB,352.743443672117,53.0691499118143,3.5736,-1.17213,1.06521,11.782,7172.0,23:30:58.43,+53:04:08.94,False,False,False,False,True,False,True,True,True,True,9,4,True,300,3,1,6,1,100,416.2,,
TIC 414026507,Kostov 2022 arXiv:2202.05790,QuadEB,336.837716720964,56.7403620051295,-3.66449,-2.95647,0.242427,10.899,9187.0,22:27:21.05,+56:44:25.30,False,True,False,True,True,False,True,True,True,False,5,2,True,300,3,1,6,1,100,322.0,,
TIC 470710327,Kostov 2022 arXiv:2202.05790,QuadEB,357.329052373724,61.9627867228284,0.6,-2.5,--,9.65,8986.0,23:49:18.97,+61:57:46.03,True,False,False,False,True,False,True,True,True,True,52,8,True,300,3,1,6,1,100,184.0,,
TIC 391620600,Kostov 2022 arXiv:2202.05790,QuadEB,71.6403891258743,44.7536621987808,-7.75529,2.23094,2.07298,12.772,5949.71,4:46:33.69,+44:45:13.18,False,True,True,True,True,False,True,True,True,False,9,2,True,300,3,1,6,1,100,724.8,,
TIC 89278612,Kostov 2022 arXiv:2202.05790,QuadEB,301.21949809707,32.6430514778645,-0.48521,-1.50692,0.663315,11.235,8740.0,20:04:52.68,+32:38:34.99,True,False,False,False,True,False,True,True,True,True,19,6,True,300,3,1,6,1,100,395.2,,
TIC 278352276,Kostov 2022 arXiv:2202.05790,QuadEB,307.503640407431,48.6070563353721,1.85387,6.57664,1.36265,10.387,7156.0,20:30:00.87,+48:36:25.40,False,True,True,True,True,False,True,True,True,False,5,4,True,300,3,1,6,1,100,202.0,,
TIC 123098844,Kostov 2022 arXiv:2202.05790,QuadEB,279.57284311725,44.6986003104844,3.4825,-9.10427,1.30925,11.136,6761.0,18:38:17.48,+44:41:54.96,False,True,False,True,True,False,True,True,True,False,15,4,True,300,3,1,6,1,100,275.0,,


* Open PIT
* Enter Title, Abstract, PIs, etc.
* Import target list from csv
* Select all targets, paste into observation tab
* Choose observing conditions
* Save XML file

In [4]:
# edit xml file to add observing times for all targets
exposure_times = {name: time / 3600 for _, (name, time) in lbt_targets[["target_name", "pepsi_exp_time"]].iterrows()}

# next, get the mapping of the nickname used in xml file to the TIC name
dir = r"..\..\Proposals\LBT\LBT 2025A"
filename = "LBT QuadEB 2025A"
tree = ET.parse(f"{dir}\\{filename}.xml")
root = tree.getroot()
target_map = {}
for target in root.findall("targets/sidereal"):
    designator = target.attrib["id"]
    name = target.find("name").text
    target_map[designator] = name

# finally, walk through each observation entry and add observing times to it
for observation in root.findall("observations/observation"):
    target_name = target_map[observation.attrib["target"]]
    target_time = exposure_times[target_name]
    time_element = ET.SubElement(observation, "time")
    time_element.attrib = {"units": "hr", "bino": "false", "too": "None", "Run": ""}
    time_element.text = f"{target_time:.3f}"

tree.write(f"{dir}\\{filename}_times.xml")